<a href="https://colab.research.google.com/github/jchen8000/DemystifyingLLMs/blob/main/4_Pre-Training/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Pre-Training

This script uses the HuggingFace datases, as a prerequisite you need a HuggingFace account and obtain a access token, see https://huggingface.co/docs/hub/security-tokens. You should add the token to Colab Secrets as HF_TOKEN.

## 4.9 Prompt Engineering on FLAN-T5

In [ ]:
%pip install transformers==4.57.3

### Load Flan-T5 Base model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def load_model(model_name):
    # Initialize the tokenizer and model from the pretrained version on Hugging Face
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model

def generate_output(tokenizer, model, input_text, max_length=200):
    # Tokenize the input text and generate the model's output
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=max_length)

    # Decode the generated tokens to a string
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

tokenizer, model = load_model('google/flan-t5-large')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
total_parameter = sum(p.numel() for p in model.parameters())
print(f"Parameters of the model: {total_parameter:,}")

Parameters of the model: 783,150,080


### Zero-shot learning

In [ ]:
prompt = f"""
Customer Review: 'The shipping was quick and the item was perfect. Totally satisfied!'.
Classification: Positive, Negative, or Neutral
"""
output = generate_output(tokenizer, model, prompt)
print(output)

Positive


### One-shot learning

In [ ]:
prompt = f"""
Customer Review: 'The restaurant was terrible, and the service was even worse. Not going back there again.'
Classification: Negative.

Customer Review: 'The new update is almost useless, the new features seem not relevant to our business flow.'
Classification: Positive, Negative, or Neutral.
"""
output = generate_output(tokenizer, model, prompt)
print(output)

Negative


### Few-shot learning

In [ ]:
prompt = f"""
Customer Review 1: 'I love this new movie, one of the best movies I've seen, definitely worth watching again!'
Classification: Positive.

Customer Review 2: 'The movie was a total waste of time and money, absolutely boring.'
Classification: Negative.

Customer Review 3: 'It's an average restaurant, the food is okay, but nothing special.'
Classification: Neutral.

Please classify the following review: 'This restaurant was clean in general, but it took a while to get my foods served.'
Classification: Positive, Negative, or Neutral.
"""
output = generate_output(tokenizer, model, prompt)
print(output)

Negative
